# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cd_weather = pd.read_csv("city_weather.csv")
cd_weather.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
555,bella union,-30.28,-57.60,50.288,70.0,0.0,3.16,UY,08-18-2020
556,pochutla,15.74,-96.47,75.992,96.0,98.0,2.11,MX,08-18-2020
557,saint-georges,12.06,-61.75,80.600,88.0,75.0,2.60,GD,08-18-2020
558,usolye-sibirskoye,52.75,103.65,62.816,50.0,50.0,4.98,RU,08-18-2020
559,casino,-28.87,153.05,71.996,30.0,0.0,2.60,AU,08-18-2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Define Latitude & Longitude as Location
locations = cd_weather[['Lat', 'Lng']].astype(float)

# Humidity as float for weight
humidity = cd_weather['Humidity'].astype(float)

In [7]:
# Create a poverty Heatmap layer
# from https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2) 

heat_layer = gmaps.heatmap_layer(
            locations, 
            weights=humidity, 
            max_intensity=100,         
            dissipating=False, 
            point_radius = 2, 
            opacity=1,
            gradient=None
                                ) # Formatting options found by shift tab

fig.add_layer(heat_layer)

plt.savefig("humidityheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrow down the DataFrame to find your ideal weather condition.
#A max temperature lower than 80 degrees but higher than 70.
sub_80 = cd_weather.loc[cd_weather["Max Temp"]<80]
sub_80_70_plus = sub_80.loc[cd_weather["Max Temp"]>70]
sub_80_70_plus_wind = sub_80_70_plus.loc[cd_weather["Wind Speed"]<10]
ideal_weather = sub_80_70_plus_wind.loc[cd_weather["Cloudiness"]==0]
ideal_weather.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
518,suez,29.97,32.53,72.752,85.0,0.0,2.91,EG,08-18-2020
528,waddan,29.16,16.14,79.034,32.0,0.0,1.76,LY,08-18-2020
533,san quintin,30.48,-115.95,75.938,67.0,0.0,2.97,MX,08-18-2020
534,bowen,-20.02,148.23,78.998,41.0,0.0,4.60,AU,08-18-2020
559,casino,-28.87,153.05,71.996,30.0,0.0,2.60,AU,08-18-2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = ""
hotel_df.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
518,suez,29.97,32.53,72.752,85.0,0.0,2.91,EG,08-18-2020,
528,waddan,29.16,16.14,79.034,32.0,0.0,1.76,LY,08-18-2020,
533,san quintin,30.48,-115.95,75.938,67.0,0.0,2.97,MX,08-18-2020,
534,bowen,-20.02,148.23,78.998,41.0,0.0,4.60,AU,08-18-2020,
559,casino,-28.87,153.05,71.996,30.0,0.0,2.60,AU,08-18-2020,


In [21]:
params = {    
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [22]:
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (KeyError, IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [17]:
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,ust-kuyga,70.02,135.60,54.824,47.0,100.0,3.20,RU,08-18-2020,
1,kanepi,57.98,26.76,50.000,87.0,100.0,1.30,EE,08-18-2020,Hobbola puhkemaja
2,bluff,-46.60,168.33,46.652,59.0,100.0,1.34,NZ,08-18-2020,Bluff Homestead - Guesthouse & Campervan Park
3,misawa,40.68,141.36,78.800,65.0,40.0,3.10,JP,08-18-2020,Hoshino Resorts Aomoriya
4,cabo san lucas,22.89,-109.91,84.992,70.0,5.0,1.61,MX,08-18-2020,Hotel Tesoro Los Cabos
5,mount gambier,-37.83,140.77,51.800,93.0,75.0,4.60,AU,08-18-2020,Southgate Motel
6,vestmannaeyjar,63.44,-20.27,53.096,92.0,98.0,2.31,IS,08-18-2020,Glamping & Camping
7,guerrero negro,27.98,-114.06,75.434,70.0,0.0,7.99,MX,08-18-2020,"Casa Laguna, Bed & Breakfast"
8,jaora,23.63,75.13,75.848,93.0,100.0,5.63,IN,08-18-2020,BLM PALACE
9,cabra,37.47,-4.44,71.006,57.0,0.0,0.64,ES,08-18-2020,MS Fuente Las Piedras


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure
fig